# Anno 1800 Savegame Visualizer

## Tabel of Contents
1. Setup
2. Customizing the Layout
3. Inspect the savegame

## Setup
**Do NOT click the button at the top to run all cells!**

This is an interactive document consisting of text parts (like this one) and code parts (with a greyish background). Click in the next line and press **Shift + Enter** to execute the code. 

In [ ]:
import importlib
if importlib.util.find_spec("lxml") is None :
    %pip install lxml

from tools.a7s_model import *

Set the language of the game. This is important for a correct decution of island names!

Paste the **path to a savegame** before you continue. Make sure to not delete the 'r' and quotation marks. Then execute the code. This may take several minutes since the file needs to be unpacked, decoded and read into memory for further processing.

In [ ]:
savegame_path = r"C:\Users\<User>\Documents\Anno 1800\accounts\<account-id>\<profile>\Autosave XXX.a7s"
try :
    world = World(Interpreter(savegame_path))
except Exception as e :
    print("ERROR:", e)

After loading the savegame, we can inspect it. We start by printing all islands to select one in the next step.

In [ ]:
world.print_island_summary()

Now set island_name to one of the names above. Ensure that upper and lower case match, the name is put into quotation marks and no additional whitespace is around the name.

In [ ]:
island_name = "Crown Falls"
island = world.get_island(island_name)
print(island)

Finally, we create an Anno Designer layout from the island and show it in Anno Designer. Make sure that **no instance of Anno Designer is open** before you proceed.

In [ ]:
show(island.get_layout())

Congretulations! You successfully exported an island. Now let's dive into it to see what more this application offers.

For example, you can re-execute a code block again with different parameters. Give it a try: Enter the name of another island and re-execute both code blocks above.

If you want to directly save the layout, you can do this too. But keep the following things in mind:
* If you do not enter an absolute path, but only a name, the file will be saved in the same directory as this application)
* Don't forget the file extension .ad

In [ ]:
save(island.get_layout(), r"test.ad")

# Customizing the Layout
All option from the graphical interface are also available here. 

In [ ]:
help(island.get_layout)

There is a more convenient way to get the desired options. Open the log console in the savegame_visualizer.ipynb. Whenever you click the "Open in Anno Designer" button the options object is printed to the console. Just copy past it here:

In [ ]:
show(island.get_layout( options = {} ))

Moreover, batch processing is easy. E.g. store all islands of the Old World session in seperate folders by their island name (you can give them a different file name than 'plan' by editing the string; if you remove the slash, all get stored in the same folder but with different names):

Hint: Valid session names are: `"The Old World", "The New World", "Kap Trelawny", "The Arctic", "Enbesa"`

In [ ]:
for isl in world.get_session("The Old World").islands.values():
    if isinstance(isl, Island):
        save(isl.get_layout(), isl.name + "/plan.ad")

# Inspect the Savegame

Let's first store the most important object to make the following code shorter:

In [ ]:
inter = world.interpreter

I have written a special method to make inspecting the savegame easier. Its purpose is twofold:
1. **Properly decode all values**: The savegame is stored as one big XML tree. However, the preprocessing tools only return an XML tree where the text of the leaf nodes is encoded hexadecimal. E.g. the GUID 180045 is encoded as DB0FC93F (hexadeximal representation of a 32-bit integer). The interpreter converts the values before displaying
2. **Give a compact overview**: The XML tree contains many arrays (e.g. for islands, objects, etc.) Displaying them in the same way as other XML editors do, this results in a very long output. The inspector instead fuses all items in an array. This has the advantage that one can see all possible child nodes. If those child nodes have different values, they are displayed as an array (e.g. ['1', '2']). For better readability, long arrays and values are truncate. The number next to a node name states how many objects were fused. If you see something like: ': 200' (i.e. the node name is missing). This means there is an array that contains both simple values (without a node name) and complex objects. Those are serialized dictionaries in the form: key1, value1, key2, value2, ...

Now let's have a look at the root of the savegame (due to preprocessing, the first method call might take some time; subsequent calls will show the result instantly):

In [ ]:
inter.inspect(world.node)

To show more, you can increase the depth to explore (don't go too deep, otherwise there is too much output and Jupyter may crash or refuse to show it)

In [ ]:
inter.inspect(world.node, 3)

You can navigate to a certain node for further inspection (search the web for 'XPath' to see how to formulate more complex queries) :

In [ ]:
inter.inspect(world.node.find("./MetaGameManager/SessionTransferManager"), 3)

However, it is cumbersome to find the correct nodes for a certain session or island. The data structure from above helps. We can get the node for each object above and inspect it further:

In [ ]:
inter.inspect(island.node, 3)

or for sessions:

In [ ]:
inter.inspect(world.get_session("The Old World").node, 3)

Some more places you might be interested to have a look at:
* `inter.inspect(world.get_session("The Old World").node.find("SessionData/BinaryData/Content/GameSessionManager/ItemSessionManager"), 5)`
* `inter.inspect(world.node.find("./MetaGameManager/EconomyStatisticManager/History/areaHistory//Snapshots"), 2)`
* `inter.inspect(world.node.find("./MetaGameManager/NewspaperManager"), 5)`
* `inter.inspect(world.node.find("./GameSetupManager/Peers"), 8)`